In [1]:
from astropy.cosmology import Planck18 as cosmo
from astropy.io import fits
import astropy.units as u

import numpy as np

import matplotlib
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})
import matplotlib.pyplot as plt

from jpasLAEs.utils import flux_to_mag

In [2]:
cat = fits.open('UV_slopes_COLA1_field.fits')[1].data
cat.columns

ColDefs(
    name = 'NUMBER_1'; format = 'D'
    name = 'ID'; format = '16A'
    name = 'groupID'; format = 'I'; null = -32768
    name = 'IS_GROUP_CHAIR'; format = 'L'
    name = 'line_fit_flag'; format = 'L'
    name = 'O3_doublet_ATT_2'; format = 'L'
    name = 'Nclumps_Y'; format = 'D'
    name = 'Nclumps_spec'; format = 'D'
    name = 'JM_comment'; format = '8A'
    name = 'ATT_comment'; format = '24A'
    name = 'SN_O3doublet'; format = 'D'
    name = 'z_O3doublet'; format = 'D'
    name = 'fluxratio_O3doublet'; format = 'D'
    name = 'ID_PARENT_det'; format = 'J'; null = -2147483648
    name = 'ALPHA_J2000_det'; format = 'D'; unit = 'deg'
    name = 'DELTA_J2000_det'; format = 'D'; unit = 'deg'
    name = 'X_IMAGE_det'; format = 'E'; unit = 'pix'
    name = 'Y_IMAGE_det'; format = 'E'; unit = 'pix'
    name = 'F115W_AUTO_mag'; format = 'E'; unit = 'mag'
    name = 'F150W_AUTO_mag'; format = 'E'; unit = 'mag'
    name = 'F200W_AUTO_mag'; format = 'E'; unit = 'mag'
    name = 'F3

In [3]:
cat_eiger = fits.open('UV_slopes_EIGER_field.fits')[1].data
cat_eiger.columns

FileNotFoundError: [Errno 2] No such file or directory: 'UV_slopes_EIGER_field.fits'

In [ ]:
def continuum_powerlaw(wavelength, norm, beta, redshift):
    return norm*(wavelength/(1500.*(1+redshift)))**beta

In [ ]:
def app_m_to_flux_nu(m):
    """
    :param m: apparent magnitude
    :return: the (observed) flux density
    """
    return 10 ** (-(m + 48.60) / 2.5)


def flux_to_lum(flux, z):
    """
    :param flux: flux
    :param z: redshift
    :return: luminosity
    """
    return flux * 4 * np.pi * cosmo.luminosity_distance(z).to("cm").value ** 2

def flux_nu_to_lum_nu(flux_nu, z):
    """
    :param flux_nu: the (observed) flux density
    :param z: redshift
    :return: the (intrinsic) luminosity density
    """
    return flux_to_lum(flux_nu, z) / (1 + z)

In [ ]:
N_iter = 1000

N_sources = len(cat)

M_UV = np.empty(N_sources)
M_UV_err_up = np.empty(N_sources)
M_UV_err_down = np.empty(N_sources)

L_UV = np.empty(N_sources)
L_UV_err_up = np.empty(N_sources)
L_UV_err_down = np.empty(N_sources)

xi_ion = np.empty(N_sources)
xi_ion_err_up = np.empty(N_sources)
xi_ion_err_down = np.empty(N_sources)

cHb = 4.86e-13

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat['beta'][src], cat['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm'][src], cat['norm_err'][src], size=N_iter)
    this_rand_Hb = np.random.normal(cat['f_Hb'][src], cat['f_Hb_err'][src], size=N_iter)

    this_redshift = cat['z_O3doublet_combined_n'][src]

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift)

    this_lum_dist = cosmo.luminosity_distance(this_redshift).to(u.cm).value

    this_L_Hb = this_rand_Hb * 1e-18 * this_lum_dist**2 * 4*np.pi / 0.4 * 0.47

    this_xi_ion = np.log10(this_L_Hb / this_L_UV / cHb)

    this_M_UV_percentiles = np.percentile(this_M_UV,[16, 50, 84])
    this_f_UV_percentiles = np.percentile(this_f_UV, [16, 50, 84])
    this_xi_ion_percentiles = np.percentile(this_xi_ion, [16, 50, 84])

    M_UV[src] = this_M_UV_percentiles[1]
    M_UV_err_up[src] = this_M_UV_percentiles[2] - M_UV[src]
    M_UV_err_down[src] = M_UV[src] - this_M_UV_percentiles[0]

    L_UV[src] = this_f_UV_percentiles[1]
    L_UV_err_up[src] = this_f_UV_percentiles[2] - L_UV[src]
    L_UV_err_down[src] = L_UV[src] - this_f_UV_percentiles[0]

    xi_ion[src] = this_xi_ion_percentiles[1]
    xi_ion_err_up[src] = this_xi_ion_percentiles[2] - xi_ion[src]
    xi_ion_err_down[src] = xi_ion[src] - this_xi_ion_percentiles[0]


In [ ]:
N_iter = 1000

N_sources = len(cat_eiger)

M_UV_eiger = np.empty(N_sources)
M_UV_err_up_eiger = np.empty(N_sources)
M_UV_err_down_eiger = np.empty(N_sources)

L_UV_eiger = np.empty(N_sources)
L_UV_err_up_eiger = np.empty(N_sources)
L_UV_err_down_eiger = np.empty(N_sources)

xi_ion_eiger = np.empty(N_sources)
xi_ion_err_up_eiger = np.empty(N_sources)
xi_ion_err_down_eiger = np.empty(N_sources)

cHb = 4.86e-13

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat_eiger['beta'][src], cat_eiger['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat_eiger['norm'][src], cat_eiger['norm_err'][src], size=N_iter)
    this_rand_Hb = np.random.normal(cat_eiger['f_Hb'][src], cat_eiger['f_Hb_err'][src], size=N_iter)

    this_redshift = cat_eiger['z_O3doublet_combined_n'][src]

    this_obs_1500 = 1500 * (1 + this_redshift)

    args = (this_obs_1500, this_rand_norm, this_rand_beta, this_redshift)
    this_f_UV = continuum_powerlaw(*args) * 1e-20
    
    this_distmod = cosmo.distmod(this_redshift).value

    this_m_UV = flux_to_mag(this_f_UV, this_obs_1500)
    this_m_UV[~np.isfinite(this_m_UV)] = -99

    this_M_UV = this_m_UV - this_distmod + 2.5 * np.log10(1 + this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift)

    this_L_UV = flux_nu_to_lum_nu(app_m_to_flux_nu(this_m_UV), this_redshift)

    this_lum_dist = cosmo.luminosity_distance(this_redshift).to(u.cm).value

    this_L_Hb = this_rand_Hb * 1e-18 * this_lum_dist**2 * 4*np.pi / 0.4 * 0.47

    this_xi_ion = np.log10(this_L_Hb / this_L_UV / cHb)

    this_M_UV_percentiles = np.percentile(this_M_UV,[16, 50, 84])
    this_f_UV_percentiles = np.percentile(this_f_UV, [16, 50, 84])
    this_xi_ion_percentiles = np.percentile(this_xi_ion, [16, 50, 84])

    M_UV_eiger[src] = this_M_UV_percentiles[1]
    M_UV_err_up_eiger[src] = this_M_UV_percentiles[2] - M_UV_eiger[src]
    M_UV_err_down_eiger[src] = M_UV_eiger[src] - this_M_UV_percentiles[0]

    L_UV_eiger[src] = this_f_UV_percentiles[1]
    L_UV_err_up_eiger[src] = this_f_UV_percentiles[2] - L_UV_eiger[src]
    L_UV_err_down_eiger[src] = L_UV_eiger[src] - this_f_UV_percentiles[0]

    xi_ion_eiger[src] = this_xi_ion_percentiles[1]
    xi_ion_err_up_eiger[src] = this_xi_ion_percentiles[2] - xi_ion_eiger[src]
    xi_ion_err_down_eiger[src] = xi_ion_eiger[src] - this_xi_ion_percentiles[0]


In [ ]:
fig, ax = plt.subplots()

ax.errorbar(cat['F115W_AUTO_mag'], M_UV, yerr=[M_UV_err_up, M_UV_err_down], ls='', fmt='o')

mask_COLA1 = cat['NUMBER_1'] == 9269
print(M_UV[mask_COLA1][0])
ax.errorbar(cat['F115W_AUTO_mag'][mask_COLA1], M_UV[mask_COLA1],
             yerr=[M_UV_err_up[mask_COLA1], M_UV_err_down[mask_COLA1]], ls='', fmt='*', ms=20, c='r')

ax.set_ylim(-22, -16)
ax.set_xlim(24, 30)

ax.set_ylabel(r'$M_{\rm UV}$')
ax.set_xlabel('F115W [magAB]')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(cat['F115W_AUTO_mag'], cat['beta'],
            yerr=cat['beta_err'],
            ls='', fmt='o', c='lightsteelblue', label='All', zorder=-1)

ax.errorbar(cat_eiger['F115W_AUTO_mag'], cat_eiger['beta'],
            yerr=cat_eiger['beta_err'],
            ls='', fmt='o', c='dimgray', label='All', zorder=-2)

ax.errorbar(np.nanmedian(cat['F115W_AUTO_mag']), np.nanmedian(cat['beta']),
            yerr=np.abs(np.nanmedian(cat['beta']) - np.nanpercentile(cat['beta'], [84, 16])).reshape(2, 1),
            zorder=999, fmt='D', ms=10, capsize=5, mfc='lightsteelblue', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average value', zorder=10)

ax.errorbar(np.nanmedian(cat_eiger['F115W_AUTO_mag']), np.nanmedian(cat_eiger['beta']),
            yerr=np.abs(np.nanmedian(cat_eiger['beta']) - np.nanpercentile(cat_eiger['beta'], [84, 16])).reshape(2, 1),
            zorder=999, fmt='D', ms=10, capsize=5, mfc='dimgray', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average value', zorder=12)

ax.errorbar(cat['F115W_AUTO_mag'][mask_COLA1], cat['beta'][mask_COLA1],
            yerr=cat['beta_err'][mask_COLA1],
            zorder=999, fmt='D', ms=10, capsize=5, mfc='r', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='COLA1', zorder=11)

print(f'COLA1: beta = {cat['beta'][mask_COLA1][0]:0.2} +/- {cat['beta_err'][mask_COLA1][0]:0.2f}')
print(f'All: beta = {np.nanmedian(cat['beta']):0.2} +/- {np.abs(np.nanmedian(cat['beta']) - np.nanpercentile(cat['beta'], [84, 16]))}')

ax.set_xlabel('F115W [magAB]')
ax.set_ylabel(r'$\beta_{\rm UV}$')

ax.legend(fontsize=11)

ax.set_xlim(24, 30)
ax.set_ylim(-5, 1)

plt.show()

In [ ]:
f_O3_Hbeta = cat['f_O3_5008'] + cat['f_O3_4960'] + cat['f_Hb']
f_O3_Hbeta_err = (cat['f_O3_5008_err']**2 + cat['f_O3_4960']**2 + cat['f_Hb']**2) ** 0.5

EW = np.empty(N_sources)
EW_err_up = np.empty(N_sources)
EW_err_down = np.empty(N_sources)

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat['beta'][src], cat['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat['norm'][src], cat['norm_err'][src], size=N_iter)

    this_redshift = cat['z_O3doublet_combined_n'][src]

    this_obs_O3_lambda = 5008.24 * (1 + this_redshift)

    args = (this_obs_O3_lambda, this_rand_norm, this_rand_beta, this_redshift)
    
    # Assume Hg/Hb = 0.4
    this_O3_cont = continuum_powerlaw(*args) * 0.4 / 0.47

    this_O3_Hbeta_f = np.random.normal(f_O3_Hbeta[src], f_O3_Hbeta_err[src], size=N_iter)

    this_EW_percs = np.percentile(this_O3_Hbeta_f / this_O3_cont, [16, 50, 84]) * (1 + this_redshift)

    EW[src] = this_EW_percs[1]
    EW_err_up[src] = this_EW_percs[2] - this_EW_percs[1]
    EW_err_down[src] = this_EW_percs[1] - this_EW_percs[0]


In [ ]:
f_O3_Hbeta_eiger = cat_eiger['f_O3_5008'] + cat_eiger['f_O3_4960'] + cat_eiger['f_Hb']
f_O3_Hbeta_err_eiger = (cat_eiger['f_O3_5008_err']**2 + cat_eiger['f_O3_4960']**2 + cat_eiger['f_Hb']**2) ** 0.5

EW_eiger = np.empty(N_sources)
EW_err_up_eiger = np.empty(N_sources)
EW_err_down_eiger = np.empty(N_sources)

for src in range(N_sources):
    this_rand_beta = np.random.normal(cat_eiger['beta'][src], cat_eiger['beta_err'][src], size=N_iter)
    this_rand_norm = np.random.normal(cat_eiger['norm'][src], cat_eiger['norm_err'][src], size=N_iter)

    this_redshift = cat_eiger['z_O3doublet_combined_n'][src]

    this_obs_O3_lambda = 5008.24 * (1 + this_redshift)

    args = (this_obs_O3_lambda, this_rand_norm, this_rand_beta, this_redshift)
    
    # Assume Hg/Hb = 0.4
    this_O3_cont = continuum_powerlaw(*args) * 0.4 / 0.47

    this_O3_Hbeta_f = np.random.normal(f_O3_Hbeta[src], f_O3_Hbeta_err[src], size=N_iter)

    this_EW_percs = np.percentile(this_O3_Hbeta_f / this_O3_cont, [16, 50, 84]) * (1 + this_redshift)

    EW_eiger[src] = this_EW_percs[1]
    EW_err_up_eiger[src] = this_EW_percs[2] - this_EW_percs[1]
    EW_err_down_eiger[src] = this_EW_percs[1] - this_EW_percs[0]

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(M_UV, EW, yerr=[EW_err_up, EW_err_down], ls='',
            fmt='o', c='gray', label='All [OIII] emitters')

ax.errorbar(M_UV[mask_COLA1], EW[mask_COLA1],
            yerr=[EW_err_up[mask_COLA1], EW_err_down[mask_COLA1]],
            xerr=[M_UV_err_up[mask_COLA1], M_UV_err_down[mask_COLA1]],
            ls='', fmt='D', ms=10, mfc='r', capsize=3, mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='COLA1')

print(f'COLA1: {EW[mask_COLA1][0]:0.2f} +/- {EW_err_up[mask_COLA1], EW_err_down[mask_COLA1]}')

M_UV_median = np.nanmedian(M_UV)
EW_median = np.nanmedian(EW)
print(f'All sources: EW = {EW_median:0.2f} +/- {np.abs(EW_median - np.nanpercentile(EW, [84, 16]))}')
ax.errorbar(M_UV_median, EW_median,
            yerr=np.abs(EW_median - np.nanpercentile(EW, [84, 16])).reshape(2, 1),
            xerr=np.abs(M_UV_median - np.nanpercentile(M_UV, [84, 16])).reshape(2, 1),
            zorder=999, fmt='s', ms=10, capsize=5, mfc='orange', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average value')

ax.set_ylim(0, 4000)

ax.set_xlabel(r'$M_{\rm UV}$')
ax.set_ylabel(r'EW$_0$([OIII] + H$\beta$) [\AA]')

ax.legend(fontsize=12)

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.errorbar(M_UV, xi_ion, yerr=[xi_ion_err_up, xi_ion_err_down], ls='',
            fmt='o', c='gray', label='All [OIII] emitters')

ax.errorbar(M_UV[mask_COLA1], xi_ion[mask_COLA1],
            yerr=[xi_ion_err_up[mask_COLA1], xi_ion_err_down[mask_COLA1]],
            xerr=[M_UV_err_up[mask_COLA1], M_UV_err_down[mask_COLA1]],
            ls='', fmt='D', ms=10, mfc='r', capsize=3, mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='COLA1')

print(f'COLA1: {xi_ion[mask_COLA1][0]:0.2f} +/- {xi_ion_err_up[mask_COLA1], xi_ion_err_down[mask_COLA1]}')

M_UV_median = np.nanmedian(M_UV)
xi_ion_median = np.nanmedian(xi_ion)
print(f'All sources: xi_ion = {xi_ion_median:0.2f} +/- {np.abs(xi_ion_median - np.nanpercentile(xi_ion, [84, 16]))}')
ax.errorbar(M_UV_median, xi_ion_median,
            yerr=np.abs(xi_ion_median - np.nanpercentile(xi_ion, [84, 16])).reshape(2, 1),
            xerr=np.abs(M_UV_median - np.nanpercentile(M_UV, [84, 16])).reshape(2, 1),
            zorder=999, fmt='s', ms=10, capsize=5, mfc='orange', mec='k', ecolor='k',
            elinewidth=2, capthick=2, mew=1.5, label='Average value')

ax.set_ylim(24.75, 27)

ax.set_xlabel(r'$M_{\rm UV}$')
ax.set_ylabel(r'$\log(\xi_{\rm ion} / {\rm Hz}\,{\rm erg}^{-1})$')

ax.legend(fontsize=12)

plt.show()